In [2]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix
# imports
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
# imports
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
# handle input like we're in first year and we understand functions
def input_data():
    lift = input("Enter lift: ")
    ORM = int(input('Enter 1RM for {}: '.format(lift)))
    count = int(input("Enter number of sets\n"))
    reps = []
    p1rm = []
    counter = 1 

    for i in range(0, count):
        print("Enter details for set {} : ".format(counter),'\n')
        reps.append(int(input("Enter reps: ")))
        p1rm.append(int(input("Enter %1RM: ")))
        counter = counter + 1

    # prep to put in dataframe
    program_data = {
        'lift': lift,
        '1RM': ORM,
        'reps': reps,
        '%1RM':p1rm
    }

    bar = int(input('Enter bar weight: '))

    plates_all_list = [0.25, 0.5, 1.25, 2.5, 5, 10, 15, 20, 25]
    plates_quantity = [4, 4, 6, 6, 10, 10, 10, 8, 8]

    plate_data = {'plate': plates_all_list, 
                  'available': plates_quantity}
    return program_data, bar, plate_data


# time to use some *good programming practices* and shove all this shit 
# into fucntions

# generate initial dataframes
# split out into seperate functions since the plates will stay the same but can have mulitple different programs

def create_program_df(program_data):
    # get val for lift ORM for calculating weight
    ORM_calc = program_data.get('1RM')
    # create dataframe
    df = pd.DataFrame(program_data)
    df['weight'] = df['%1RM'].apply(lambda x: x/100*ORM_calc)    
    return df

def create_plates_df(plate_data):
    # create dataframe
    plates = pd.DataFrame(plate_data)
    
    # half the available plates for calculation value, round to deal with uneven
    plates['avail_calc'] = (plates['available'].apply(lambda x: x/2)).round(0)

    # create list of available plates repeating per quantity to use in calculation function
    plates_list = list(plates['plate'].reindex(plates.index.repeat(plates.avail_calc)))
    return plates, plates_list

# calculations
# calc how many of each plate per side to get weight per set

# generator from stackoverflow that does the job
# numbers = plates list
# target = weight
def subset_sum(numbers, target, partial=[], partial_sum=0):
    if partial_sum == target:
        yield partial
    if partial_sum >= target:
        return
    for i, n in enumerate(numbers):
        remaining = numbers[i + 1:]
        yield from subset_sum(remaining, target, partial + [n], partial_sum + n)

# create dataframe with the plate combinations, narrow down to top 5 per weight based on least number of plates needed
def generate_combinations(program_df, plates_list, bar):
    # get list of weights from program dataframe, round from float to int
    list_weights = [round(x) for x in list(program['weight'])]
    # to get the amount needed per side, remove the bar weight and half the total
    list_weights = [(x-bar)/2 for x in list_weights]

    # create dictionary and call generator to populate using weight as key
    d = {}
    for i, val in enumerate(list_weights):
        newlist = list(subset_sum(plates_list, val))
        key = val
        d[key] = newlist 

    # build dataframe from dictionary
    df = pd.DataFrame(list(d.items()),columns=['weight','combinations'])
    # explode to get each combination per row
    df = df.explode('combinations').set_index(['weight'], append=True)

    # deal with negative values (if weight is less than the bar)
    # split dataframe into 0 values
    df_na = df[(df['combinations'].isna()) | (df["combinations"].str.len() == 0)]
    df_na = df_na.reset_index(level=1)
    # assign string based on weight
    df_na['combinations'] = np.where(
        df_na['weight'] == 0, 'bar weight only', np.where(
        df_na['weight'] <  0, 'weight under bar weight', 'err')) 

    # non-na df
    df = df[(df['combinations'].notna())]
    df = df[df['combinations'].map(lambda d: len(d)) > 0]

    # convert lists to strings seperated by ;
    df['combinations'] = ['; '.join(map(str, l)) for l in df['combinations']]

    # drop duplicates & remove the multi-index
    df = df.drop_duplicates()

    df = df.reset_index(level=1)

    # rejoin df_na to df now that combinations is a string column
    df = pd.concat([df, df_na], axis=0)

    # narrow down dataframe
    # get best combination (i.e. lowest number of elements)
    df['len'] = df['combinations'].str.count(';')

    # sort into top 5 options 
    df = df.sort_values('len', ascending=True).groupby(['weight']).head(5)
    df = df.sort_values(['weight','len']).drop(['len'], axis=1)

    # join back to original dataframe
    # recalculate the required weight to join back 
    final = df.rename(columns={'weight':'plate_weight','combinations':'plates' })
    # add bar weight and * 2
    final['weight'] = final['plate_weight'].apply(lambda x: (x*2)+bar)

    # merge onto program dataframe
    final = program_df.merge(final, on='weight', how='left')
    final.sort_values(['reps','%1RM','weight'], ascending=False)
    return final

In [7]:
# test the functions
program_data, bar, plate_data = input_data()
program = create_program_df(program_data)
plates, plates_list = create_plates_df(plate_data)
final = generate_combinations(program, plates_list,bar)
print(final)

Enter lift: Bench
Enter 1RM for Bench: 50
Enter number of sets
3
Enter details for set 1 :  

Enter reps: 4
Enter %1RM: 65
Enter details for set 2 :  

Enter reps: 3
Enter %1RM: 70
Enter details for set 3 :  

Enter reps: 5
Enter %1RM: 80
Enter bar weight: 20
     lift  1RM  reps  %1RM  weight  plate_weight                     plates
0   Bench   50     4    65    32.5           NaN                        NaN
1   Bench   50     3    70    35.0           7.5                   2.5; 5.0
2   Bench   50     3    70    35.0           7.5              2.5; 2.5; 2.5
3   Bench   50     3    70    35.0           7.5            1.25; 1.25; 5.0
4   Bench   50     3    70    35.0           7.5       1.25; 1.25; 2.5; 2.5
5   Bench   50     3    70    35.0           7.5  0.25; 0.5; 0.5; 1.25; 5.0
6   Bench   50     5    80    40.0          10.0                       10.0
7   Bench   50     5    80    40.0          10.0                   5.0; 5.0
8   Bench   50     5    80    40.0          10.0        